
# 🧠 Session 2: Train Your Own Image Classifier — Instructor Edition

Welcome, instructor! 🎓  
This version of the notebook contains additional notes, teaching guidance, real‑world connections, and example solutions.  
It follows the same structure as the student notebook but with *💡 Instructor Tips*, *⚠️ Pay Attention*, and *🌍 Real‑World Examples* embedded throughout.



# 🧠 Session 2: Train Your Own Image Classifier (Beginner Level)

Welcome to **Session 2** of Newegg’s *AI Foundations Workshop*! 🎓  
In this session, you’ll **teach an AI to recognize images** — just like how your phone camera knows what’s a cat or a car!  
We’ll use **PyTorch** and a fun dataset called **CIFAR‑10**, which contains 10 types of small color images.

By the end, you’ll have a mini AI model that can recognize images — a key skill for building your final **AI Game** in Session 4. 🎮



## 🎯 What You’ll Learn
- How image classification works  
- How to train and test a simple AI model  
- How to visualize and understand model predictions  



## ⚙️ Step 1: Setup Your Environment
Let’s import the libraries we need. If any are missing, you can install them by running:
```bash
pip install torch torchvision matplotlib
```


In [ ]:

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

print("PyTorch version:", torch.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)



## 🖼️ Step 2: Load and Explore the CIFAR‑10 Dataset
The CIFAR‑10 dataset has 60,000 color images in 10 classes (airplane, car, bird, cat, etc.).  
Each image is only 32×32 pixels — perfect for quick training! 🚀


In [ ]:

# Transform: convert images to tensors and normalize them
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Download and load training and test datasets
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=8, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Helper function to show images
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.figure(figsize=(6,2))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis('off')
    plt.show()

# Get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# Show images
imshow(torchvision.utils.make_grid(images))
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(8)))



## 🧩 Step 3: Build a Simple Neural Network
We’ll build a small **Convolutional Neural Network (CNN)** — a type of model that’s great for recognizing images.


In [ ]:

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net().to(device)
print(net)



## ⚙️ Step 4: Define Loss Function and Optimizer
The **loss function** measures how wrong the AI is, and the **optimizer** helps it learn from mistakes.


In [ ]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)



## 🚀 Step 5: Train the Classifier
Now we’ll train the AI using the training images.  
This will take a few minutes depending on your computer.


In [ ]:

for epoch in range(2):  # run for 2 epochs for demo
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 1000 == 999:
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 1000:.3f}')
            running_loss = 0.0

print('✅ Training complete!')



## 🔍 Step 6: Test the Model and See How It Performs
Let’s check how well our model learned to recognize images!


In [ ]:

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on 10,000 test images: {100 * correct / total:.2f}%')



## 👀 Step 7: Visualize Model Predictions
Let’s see what our AI predicts for a few test images!


In [ ]:

dataiter = iter(testloader)
images, labels = next(dataiter)

imshow(torchvision.utils.make_grid(images))
print('GroundTruth:', ' '.join(f'{classes[labels[j]]:5s}' for j in range(8)))

outputs = net(images.to(device))
_, predicted = torch.max(outputs, 1)

print('Predicted:  ', ' '.join(f'{classes[predicted[j]]:5s}' for j in range(8)))



## 🧪 Step 8: Try Your Own Image! (Optional)
You can upload your own small image and let the AI guess what it is.  
Tip: The image should look like one of the 10 categories in CIFAR‑10!


In [ ]:

from PIL import Image
import requests

url = 'https://raw.githubusercontent.com/pytorch/hub/master/images/dog.jpg'  # example
img = Image.open(requests.get(url, stream=True).raw).resize((32, 32))
plt.imshow(img)
plt.axis('off')
plt.show()

transform_single = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
img_tensor = transform_single(img).unsqueeze(0).to(device)

net.eval()
output = net(img_tensor)
_, predicted = torch.max(output, 1)
print("AI thinks this is a:", classes[predicted[0]])



## 🎉 Step 9: Wrap-Up & What’s Next
Amazing work! You just trained your **first image classification model**. 🏆  
In the next session, you’ll learn how to build a **Chatbot** 🤖 that can talk —  
and later combine it with your image AI to make your **AI Guessing Game**! 🎮



## 🧩 Level Up! – Mini Challenges

Great job finishing your image classifier! 🎉  
Now, let’s explore and experiment a little more. Each challenge below helps you think like an AI engineer. 👩‍💻👨‍💻

### 1️⃣ 🧠 Train Longer
Increase the number of **epochs** in the training loop (for example, from `2` to `5` or `10`)  
👉 Does the accuracy improve? Does the training take longer?

### 2️⃣ 🎨 Modify the Model
Try changing the neural network structure — for example:  
- Add another convolutional layer  
- Change the number of filters (e.g., from 6 to 8)  
👉 Observe how these changes affect performance.

### 3️⃣ ⚙️ Tune the Training Parameters
Experiment with the **learning rate** or **batch size**:  
```python
optimizer = optim.SGD(net.parameters(), lr=0.005, momentum=0.9)
```
👉 What happens if you make the learning rate smaller or larger?

### 4️⃣ 📸 Use Your Own Images
Collect 2–3 small photos from your phone or the internet (32×32 pixels recommended).  
Try running them through the model — does your AI guess correctly?

### 5️⃣ 🚀 Bonus Challenge – Advanced Dataset
Replace CIFAR‑10 with **CIFAR‑100**, which has 100 categories instead of 10!  
Hint: You only need to change one line in the dataset loading section.  
👉 Can your network handle it?

---

These experiments will prepare you for **Session 4**, where you’ll combine everything into a creative **AI Guessing Game! 🎮**



---
# 💡 Instructor Guidance & Enrichment Notes

### 🧭 Session Overview
Students will learn to train an image classifier using PyTorch and the CIFAR‑10 dataset.  
Encourage curiosity and creativity — focus on **how machines learn patterns** rather than coding perfection.

---
## 🧩 Section-by-Section Instructor Notes

### ⚙️ Step 1 – Setup
💡 **Tip:** If internet speed is slow, pre‑download the CIFAR‑10 dataset and store it locally on workshop machines.  
⚠️ **Common Issue:** Students may run into `RuntimeError: CUDA not available`. Encourage them to switch to CPU mode — training will be slower but fine.

---
### 🖼️ Step 2 – Explore the Dataset
🌍 **Real‑World Example:** Explain that image classification is the foundation for technologies like **Google Photos’ search** (“show me cats”) or **self‑driving car vision systems**.  
🎉 **Fun Idea:** Let students vote which class (e.g., cat vs. dog) they think the AI will confuse most!

---
### 🧩 Step 3 – Build the Neural Network
💡 **Tip:** Use the analogy of the **human brain** — convolutional layers act like *eyes* detecting shapes and edges, while fully connected layers are the *decision-makers*.  
⚠️ **Common Issue:** If students modify the number of layers or neurons, remind them to adjust the linear layer input size accordingly.

---
### 🚀 Step 5 – Train the Model
💡 **Tip:** Encourage students to notice the *loss decreasing*. That’s like the AI “getting better at learning.”  
🌍 **Real‑World Connection:** Talk about **how AI models are trained on massive image datasets** (e.g., ImageNet) and how this small CIFAR‑10 example is a simplified version of real‑world AI training.

---
### 🔍 Step 6 – Evaluate Performance
🎉 **Fun Activity:** Have students compare their test accuracy. Make a leaderboard!  
💡 **Tip:** Discuss **overfitting** in simple terms: when an AI memorizes instead of generalizing.

---
### 🧪 Step 8 – Try Your Own Image
🌍 **Example:** Explain how phone apps like Snapchat filters or Amazon product scanners work using similar classification logic.  
💡 **Tip:** If bandwidth allows, let students bring an image of a real object and see what the AI thinks!

---
## 🏆 Challenge Solutions & Discussion

### 1️⃣ 🧠 Train Longer
✅ **Answer:** Accuracy usually improves slightly (e.g., from 55% → 60%) with more epochs, but overfitting may appear after 10+.  
💬 *Teaching Point:* More training ≠ always better — balance is key.

### 2️⃣ 🎨 Modify the Model
✅ **Example:** Adding another conv layer (`nn.Conv2d(16, 32, 3)`) can boost accuracy, but increases training time.  
💬 *Teaching Point:* More layers help the model “see” more complex features, but make sure the dataset is large enough.

### 3️⃣ ⚙️ Tune Parameters
✅ **Example:** Smaller learning rate (e.g., 0.0005) → slower learning but more stable.  
Larger rate (e.g., 0.01) → faster but may overshoot.  
💬 *Teaching Point:* Hyperparameter tuning is like “finding the sweet spot” — there’s no single correct value.

### 4️⃣ 📸 Use Your Own Images
✅ **Solution:** Convert custom images to 32×32 pixels, normalize, then test. Some images may give funny or wrong results — that’s normal!  
💬 *Teaching Point:* Discuss why context matters — the AI only learned from 10 object categories.

### 5️⃣ 🚀 Bonus Challenge – CIFAR‑100
✅ **Answer:** The same code works with `torchvision.datasets.CIFAR100`. Accuracy will drop significantly (~20%) since it’s a much harder task.  
💬 *Teaching Point:* Real‑world AI often faces “CIFAR‑100 problems” — vast diversity, limited data.

---
## 🎓 Wrap-Up
🎯 Reinforce key ideas:
- AI “learns from examples.”  
- Model performance depends on **data quality + architecture + tuning.**  
- Encourage exploration — that’s how great data scientists start!

🎉 **Fun Closing Idea:** Have students describe one real-world product that likely uses image classification (e.g., Tesla Autopilot, TikTok filters, airport scanners).  
